In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import statsmodels
import scipy
from scipy import stats
from statsmodels.stats.weightstats import ttest_ind
import requests
import mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv(r'C:\Users\matti\OneDrive\Desktop\New folder\Data Model - Pizza Sales.csv')

In [4]:
df

,order_details_id,order_id,pizza_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,01/01/2015,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,01/01/2015,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,01/01/2015,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4,2,ital_supr_l,1,01/01/2015,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5,2,mexicana_m,1,01/01/2015,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza
...,...,...,...,...,...,...,...,...,...,...,...,...
48615,48616,21348,ckn_alfredo_m,1,31/12/2015,21:23:10,16.75,16.75,M,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A...",The Chicken Alfredo Pizza
48616,48617,21348,four_cheese_l,1,31/12/2015,21:23:10,17.95,17.95,L,Veggie,"Ricotta Cheese, Gorgonzola Piccante Cheese, Mo...",The Four Cheese Pizza
48617,48618,21348,napolitana_s,1,31/12/2015,21:23:10,12.00,12.00,S,Classic,"Tomatoes, Anchovies, Green Olives, Red Onions,...",The Napolitana Pizza
48618,48619,21349,mexicana_l,1,31/12/2015,22:09:54,20.25,20.25,L,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48620 entries, 0 to 48619
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_details_id   48620 non-null  int64  
 1   order_id           48620 non-null  int64  
 2   pizza_id           48620 non-null  object 
 3   quantity           48620 non-null  int64  
 4   order_date         48620 non-null  object 
 5   order_time         48620 non-null  object 
 6   unit_price         48620 non-null  float64
 7   total_price        48620 non-null  float64
 8   pizza_size         48620 non-null  object 
 9   pizza_category     48620 non-null  object 
 10  pizza_ingredients  48620 non-null  object 
 11  pizza_name         48620 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 4.5+ MB


In [6]:
df.isnull().sum()

order_details_id     0
order_id             0
pizza_id             0
quantity             0
order_date           0
order_time           0
unit_price           0
total_price          0
pizza_size           0
pizza_category       0
pizza_ingredients    0
pizza_name           0
dtype: int64

In [7]:
import re

REMOVE EXTRA SPACE IN THE PIZZA ID 

In [8]:
df['pizza_id']=df['pizza_id'].astype('str').apply(lambda x: re.sub(r'\s+',' ',x).strip())

In [9]:
df['order_id'].unique()

array([    1,     2,     3, ..., 21348, 21349, 21350], dtype=int64)

In [ ]:
#REMOVE EXTRA SPACE IN THE ORDER ID 

In [10]:
df['order_id']=df['order_id'].astype('str').apply(lambda x: re.sub(r'\s+',' ',x).strip())

In [ ]:
#CONVERT ORDER ID TO NUMERIC

In [11]:
df['order_id']=pd.to_numeric(df['order_id'])

In [12]:
df['order_id'].unique()

array([    1,     2,     3, ..., 21348, 21349, 21350], dtype=int64)

In [13]:
print('total number of pizza:',df['pizza_id'].nunique()) 
df['pizza_id'].value_counts(normalize=True)

total number of pizza: 91


pizza_id
big_meat_s        0.037248
thai_ckn_l        0.028075
five_cheese_l     0.027951
four_cheese_l     0.026183
classic_dlx_m     0.023838
                    ...   
mexicana_s        0.003291
calabrese_s       0.002036
ckn_alfredo_s     0.001974
green_garden_l    0.001933
the_greek_xxl     0.000576
Name: proportion, Length: 91, dtype: float64

In [14]:
grouped_df2=(df
             .groupby(['order_id','pizza_id'])  #group the results per invoice number and product type - andremo a vedere in ogni invoice quali prodotti ci sono
        .size() #fornisce il count delle invoice
        .unstack() #va a pivotare Product line in colonne
        .fillna(0) # visto che tutti i prodotti vengono pivotati in colonna nelle invoice dove un prodotto e' assente il sistema dara NaN - fillna converte il Nan in 0
        .reset_index() # andiamo ora a mutare Invoice ID in index
         .set_index('order_id'))

In [15]:
grouped_df2

pizza_id,bbq_ckn_l,bbq_ckn_m,bbq_ckn_s,big_meat_s,brie_carre_s,calabrese_l,calabrese_m,calabrese_s,cali_ckn_l,cali_ckn_m,...,thai_ckn_m,thai_ckn_s,the_greek_l,the_greek_m,the_greek_s,the_greek_xl,the_greek_xxl,veggie_veg_l,veggie_veg_m,veggie_veg_s
order_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21346,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21347,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
grouped_df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21350 entries, 1 to 21350
Data columns (total 91 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bbq_ckn_l       21350 non-null  float64
 1   bbq_ckn_m       21350 non-null  float64
 2   bbq_ckn_s       21350 non-null  float64
 3   big_meat_s      21350 non-null  float64
 4   brie_carre_s    21350 non-null  float64
 5   calabrese_l     21350 non-null  float64
 6   calabrese_m     21350 non-null  float64
 7   calabrese_s     21350 non-null  float64
 8   cali_ckn_l      21350 non-null  float64
 9   cali_ckn_m      21350 non-null  float64
 10  cali_ckn_s      21350 non-null  float64
 11  ckn_alfredo_l   21350 non-null  float64
 12  ckn_alfredo_m   21350 non-null  float64
 13  ckn_alfredo_s   21350 non-null  float64
 14  ckn_pesto_l     21350 non-null  float64
 15  ckn_pesto_m     21350 non-null  float64
 16  ckn_pesto_s     21350 non-null  float64
 17  classic_dlx_l   21350 non-null  floa

In [17]:
#andiamo ad assegnare 1 se il prodotto e' presente
def market_basket(x):
    if x <=0:
        return 0
    if x >= 1:
        return 1 
    
grouped_df2=grouped_df2.applymap(market_basket)

In [24]:
frequent_interest=apriori(grouped_df2,min_support=0.001,use_colnames=True) 
# SUPPORTO ---> Questo parametro specifica il supporto minimo richiesto per un itemset per essere considerato "frequente". 
#Il supporto è la frazione di transazioni nel dataset che contiene l'itemset. 
#Un min_support di 0.001 significa che un itemset deve apparire in almeno lo 0.1% di tutte le transazioni per essere considerato frequente.

MemoryError: Unable to allocate 14.0 GiB for an array with shape (29422, 3, 21350) and data type int64

In [20]:
frequent_interest

,support,itemsets
0,0.045293,(bbq_ckn_l)
1,0.043372,(bbq_ckn_m)
2,0.022436,(bbq_ckn_s)
3,0.084824,(big_meat_s)
4,0.022482,(brie_carre_s)
...,...,...
78,0.014052,(the_greek_s)
79,0.025480,(the_greek_xl)
80,0.019813,(veggie_veg_l)
81,0.029508,(veggie_veg_m)


In [21]:
rules=association_rules(frequent_interest,metric="confidence",min_threshold=0.1)
#CONFIDENCE-Questo parametro specifica la metrica utilizzata per filtrare le regole generate.
# La confidence (fiducia) è la probabilità condizionata che l'item B sia presente nelle transazioni che contengono l'item A, cioè confidence(A -> B) = support(A ∪ B) / support(A). 
#In questo caso, le regole generate saranno filtrate in base alla fiducia.

In [22]:
if not frequent_interest.empty:
    rules = association_rules(frequent_interest, metric="confidence", min_threshold=0)
    print("Association Rules:")
    print(rules)
else:
    print("No frequent itemsets found.")

Association Rules:
Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction, zhangs_metric]
Index: []


In [23]:
rules.sort_values(by='lift',ascending=False)
#lift: È una misura di quanto l'antecedente (A) aumenta la probabilità del conseguente (B). 
#Se il lift è maggiore di 1, significa che A e B sono correlati positivamente (la presenza di A aumenta la probabilità di B).
# Un lift inferiore a 1 indica una correlazione negativa, mentre un lift uguale a 1 indica indipendenza.

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
